**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)
            
        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

*The function act chooses the next action to do. It returns a random action with probability ```epsilon``` and it returns the action with respect to the policy with probability ```1-epsilon ```. ```epsilon``` is essential because it controls the eploration-exploitation dilemma : if ```epsilon``` is too small, the agent will do little exploration and thus, will unlikely determine the optimal policy. If ```epsilon``` is too high, the agent will rarely follow the policy.*

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=11 # set small when debugging
epochs_test=5 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

*```position``` contains the postion of the rat in the current state. The cell where it is located contains a 1, those that contain -1 are those where it cannot go. We can note that 2 lines and columns of -1 were added at the begining and the ending of the position and board grid to ensure that the rat sees a range of 2 cells.*

*```board``` contains the reward that the rat would get if it went on each cell.*

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        #pick a random action between 0 and 3
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        #starting state
        state = env.reset()

        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # get random action
            action = agent.act(state)
            # perform action and look at the results
            state, reward, game_over = env.act(action)

            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.5/21.0. Average score (-8.5)
Win/lose count 11.0/7.0. Average score (-2.25)
Win/lose count 9.5/7.0. Average score (-0.6666666666666666)
Win/lose count 12.5/15.0. Average score (-1.125)
Win/lose count 11.0/12.0. Average score (-1.1)
Final score: -1.1


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**1)** We have that: 
$\begin{array}
QQ^{\pi}(s,a) & = \mathbb{E}_{p^\pi}[\sum_{t\geq 0}\gamma^t r(s_t, a_t)|s_0=s,a_0=a] 
\end{array}$

As we are in a Markov chain, we can write it as:

$\begin{array}
QQ^{\pi}(s,a) & = r(s,a) + \sum_{a',s'}p(a',s'|s_0=s,a_0=a)\mathbb{E}[\sum_{t\geq 1}\gamma^t r(s_t, a_t)|s_1=s',a_1=a'] 
\end{array}$

We can factorize by $\gamma$ and make the variable change $t'=t-1$ and we get:

$\begin{array}
QQ^{\pi}(s,a) & = r(s,a) + \sum_{a',s'}p(a',s'|s_0=s,a_0=a)\gamma Q^{\pi}(s',a')\\
& = \mathbb{E}_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')]
\end{array}$

**2)** We have:

$\begin{array}QQ^*(s,a) & =\underset{\pi}{\max} Q^{\pi}(s,a)\\
& = r(s,a) + \gamma \underset{\pi}{\max} \mathbb{E}_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]
\end{array}$

We also have:

$\begin{array}
k\max_{\pi}\mathbb{E}_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] & \leq \mathbb{E}_{s'\sim \pi^*(.|s,a)}[\underset{a'}{\max}\underset{\pi}{\max}Q^{\pi}(s',a')]\\
 & \leq \mathbb{E}_{s'\sim \pi^*(.|s,a)}[\underset{a'}{\max}Q^{*}(s',a')]
\end{array}$

And:

$\begin{array}
k\mathbb{E}_{s'\sim \pi^*(.|s,a)}[\underset{a'}{\max}Q^{*}(s',a')] & = \underset{a'}{\max} \mathbb{E}_{s'\sim \pi^*(.|s,a)}[Q^{*}(s',a')] \\
& \leq \underset{a'}{\max} \underset{s'}{\max}\mathbb{E}_{(s',a')\sim p(.|s,a)}[Q^{\pi^*}(s',a')] \\
& \leq \underset{\pi}{\max} \mathbb{E}_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] 
\end{array}$

Thus, we have

$ \underset{\pi}{\max} \mathbb{E}_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]  = \mathbb{E}_{s'\sim \pi^*(.|s,a)}[\underset{a'}{\max}Q^{*}(s',a')]$

Hence, we get the equality:

$$ Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].$$

**3)** The optimal policy verifies the above equality, therefore we want to have the two parts the closest possible with our Q-function wich takes $\theta$ as parameter. By minimizing the following loss, we can approach it :

$\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if (len(self.memory) <= self.max_memory):
            self.memory.append(m)
        else:
            #remove the first element and add m at the end of the list
            self.memory.pop(0)
            self.memory.append(m)

    def random_access(self):
        n = len(self.memory)
        index = np.random.randint(n)
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        

    def learned_act(self, s):
        # the prediction is the argmax of the ouput of the model
        return np.argmax(self.model.predict(s.reshape(1,5,5,self.n_state)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i] = s
            #get the of the model and store them
            target_q[i] = self.model.predict(s.reshape(1,5,5,self.n_state))
            
            if game_over:
                target_q[i][a] = r
                
            else:
                target_q[i][a] = r + self.discount * np.max(self.model.predict(n_s.reshape(1,5,5,self.n_state)))
                
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential([
            Flatten(),
            Dense(64, input_shape=(5*5*self.n_state,), activation = 'relu'),
            Dense(32, activation='relu'),
            Dense(4)]
        )
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.2, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, 31, prefix='fc_train')
HTML(display_videos('fc_train30.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 000/031 | Loss 0.0210 | Win/lose count 2.0/4.0 (-2.0)
Epoch 001/031 | Loss 0.0088 | Win/lose count 1.0/1.0 (0.0)
Epoch 002/031 | Loss 0.0436 | Win/lose count 5.5/6.0 (-0.5)
Epoch 003/031 | Loss 0.0069 | Win/lose count 3.0/2.0 (1.0)
Epoch 004/031 | Loss 0.0538 | Win/lose count 3.0/3.0 (0.0)
Epoch 005/031 | Loss 0.0031 | Win/lose count 1.5/9.0 (-7.5)
Epoch 006/031 | Loss 0.0071 | Win/lose count 5.0/6.0 (-1.0)
Epoch 007/031 | Loss 0.0022 | Win/lose count 4.0/4.0 (0.0)
Epoch 008/031 | Loss 0.0133 | Win/lose count 3.5/5.0 (-1.5)
Epoch 009/031 | Loss 0.0028 | Win/lose count 2.5/4.0 (-1.5)
Epoch 010/031 | Loss 0.0089 | Win/lose count 3.5/4.0 (-0.5)
Epoch 011/031 | Loss 0.0040 | Win/lose count 4.0/3.0 (1.0)
Epoch 012/031 | Loss 0.0042 | Win/lose count 2.5/5.0 (-2.5)
Epoch 013/031 | Loss 0.0049 | Win/lose count 2.0/2.0 (0.0)
Epoch 014/031 | Loss 0.0402 | Win/lose count 3

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential([
                    Conv2D(40, (2,2), input_shape=(5,5,self.n_state), padding='same'),
                    Activation('relu'),
                    Conv2D(20, (2,2), padding='same'),
                    Activation('relu'),
                    Flatten(),
                    Dense(4)]
                )        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.2, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent,env,31,prefix='cnn_train')
HTML(display_videos('cnn_train30.mp4'))

Epoch 000/031 | Loss 0.0033 | Win/lose count 4.5/6.0 (-1.5)
Epoch 001/031 | Loss 0.0293 | Win/lose count 3.5/2.0 (1.5)
Epoch 002/031 | Loss 0.0068 | Win/lose count 5.5/2.0 (3.5)
Epoch 003/031 | Loss 0.0292 | Win/lose count 7.0/3.0 (4.0)
Epoch 004/031 | Loss 0.0011 | Win/lose count 11.0/5.0 (6.0)
Epoch 005/031 | Loss 0.0293 | Win/lose count 6.0/5.0 (1.0)
Epoch 006/031 | Loss 0.0036 | Win/lose count 9.0/4.0 (5.0)
Epoch 007/031 | Loss 0.0018 | Win/lose count 10.5/4.0 (6.5)
Epoch 008/031 | Loss 0.0043 | Win/lose count 8.5/3.0 (5.5)
Epoch 009/031 | Loss 0.0016 | Win/lose count 3.0/0 (3.0)
Epoch 010/031 | Loss 0.0052 | Win/lose count 9.5/10.0 (-0.5)
Epoch 011/031 | Loss 0.0024 | Win/lose count 11.0/5.0 (6.0)
Epoch 012/031 | Loss 0.0323 | Win/lose count 12.5/2.0 (10.5)
Epoch 013/031 | Loss 0.0277 | Win/lose count 7.0/1.0 (6.0)
Epoch 014/031 | Loss 0.0274 | Win/lose count 14.0/3.0 (11.0)
Epoch 015/031 | Loss 0.0038 | Win/lose count 10.5/4.0 (6.5)
Epoch 016/031 | Loss 0.0266 | Win/lose count 12

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [20]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
epochs_test = 20
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.0/4.0. Average score (-1.0)
Win/lose count 4.0/1.0. Average score (1.0)
Win/lose count 5.5/2.0. Average score (1.8333333333333333)
Win/lose count 16.0/4.0. Average score (4.375)
Win/lose count 7.5/3.0. Average score (4.4)
Win/lose count 17.5/5.0. Average score (5.75)
Win/lose count 22.0/5.0. Average score (7.357142857142857)
Win/lose count 20.0/1.0. Average score (8.8125)
Win/lose count 10.0/2.0. Average score (8.722222222222221)
Win/lose count 7.5/2.0. Average score (8.4)
Win/lose count 13.5/4.0. Average score (8.5)
Win/lose count 7.5/2.0. Average score (8.25)
Win/lose count 4.0/3.0. Average score (7.6923076923076925)
Win/lose count 3.0/3.0. Average score (7.142857142857143)
Win/lose count 8.5/4.0. Average score (6.966666666666667)
Win/lose count 11.5/5.0. Average score (6.9375)
Win/lose count 11.5/4.0. Average score (6.970588235294118)
Win/lose count 10.0/4.0. Average score (6.916666666666667)
Win/lose count 9.5/4.0. Average score (6.842105263157895)


In [21]:
HTML(display_videos('cnn_test0.mp4'))

In [22]:
HTML(display_videos('fc_test0.mp4'))

*Le modèle de réseau convolutionels semble mieux que le modèle fully-connected. On peut remarquer qu'en moyenne, le réseau convolutif a des meilleurs résultats que le réseau fully-connected et que pendant l'entrainement, les scores sont bien mieux pour le cnn.*

*Le réseau convolutif semble prendre moins de risque que l'autre modèle. En effet, lorsque l'on met une faible température, c'est à dire lorsqu'il y a peur de cases avec des récompenses, on observe que le cnn prends peu de mauvaise récompense alors que l'autre en recoit plus. En revanche, avec une température égale à 1, lorsque la grille est recouverte uniquement de récompenses positives, on voit que le réseau fully-connected explore plus que le cnn, ce qui lui donne de meilleurs scores. *

*Le problème principal de ces deux réseaux est qu'ils ont tendance à rester sur place et à très peu explorer. En effet, dans ces cas là il ont une récompense nulle mais comme il se peu que l'état présent n'affiche pas de récompense positive, l'algorithme ne se risque pas à explorer et fait des aller-retour entre deux case.*

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch, decay_factor = 0.9, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(agent.epsilon*decay_factor)
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        #prise en compte du malus de position
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        #on ré-initialise malus position à zero partout, sauf la position actuelle à 0.1
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x,self.y] = 0.1

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    


In [24]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.6, memory_size=2000, batch_size = 64,n_state=3)
train_explore(agent, env, 51, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

Epoch 000/051 | Loss 0.0199 | Win/lose count 9.0/29.10000000000008 (-20.10000000000008)
Epoch 001/051 | Loss 0.0169 | Win/lose count 5.5/24.100000000000055 (-18.600000000000055)
Epoch 002/051 | Loss 0.0069 | Win/lose count 10.5/17.399999999999984 (-6.899999999999984)
Epoch 003/051 | Loss 0.0091 | Win/lose count 9.5/19.90000000000002 (-10.40000000000002)
Epoch 004/051 | Loss 0.0085 | Win/lose count 9.0/22.700000000000014 (-13.700000000000014)
Epoch 005/051 | Loss 0.0145 | Win/lose count 16.5/16.49999999999997 (2.842170943040401e-14)
Epoch 006/051 | Loss 0.0036 | Win/lose count 13.0/23.700000000000003 (-10.700000000000003)
Epoch 007/051 | Loss 0.0087 | Win/lose count 8.5/20.50000000000001 (-12.00000000000001)
Epoch 008/051 | Loss 0.0276 | Win/lose count 15.5/19.299999999999994 (-3.7999999999999936)
Epoch 009/051 | Loss 0.0468 | Win/lose count 21.5/19.29999999999999 (2.20000000000001)
Epoch 010/051 | Loss 0.0047 | Win/lose count 18.0/18.2 (-0.1999999999999993)
Epoch 011/051 | Loss 0.0056 

In [25]:
# Evaluation
epochs_test = 21
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore20.mp4'))

Win/lose count 17.5/0. Average score (17.5)
Win/lose count 18.0/1.0. Average score (17.25)
Win/lose count 17.5/4.0. Average score (16.0)
Win/lose count 5.5/1.0. Average score (13.125)
Win/lose count 23.5/3.0. Average score (14.6)
Win/lose count 13.5/3.0. Average score (13.916666666666666)
Win/lose count 13.0/1.0. Average score (13.642857142857142)
Win/lose count 7.0/0. Average score (12.8125)
Win/lose count 21.0/2.0. Average score (13.5)
Win/lose count 16.0/1.0. Average score (13.65)
Win/lose count 17.5/2.0. Average score (13.818181818181818)
Win/lose count 13.0/1.0. Average score (13.666666666666666)
Win/lose count 9.0/0. Average score (13.307692307692308)
Win/lose count 10.5/1.0. Average score (13.035714285714286)
Win/lose count 19.0/5.0. Average score (13.1)
Win/lose count 12.5/1.0. Average score (13.0)
Win/lose count 2.0/0. Average score (12.352941176470589)
Win/lose count 7.0/0. Average score (12.055555555555555)
Win/lose count 20.5/1.0. Average score (12.447368421052632)
Win/lose

À présent, l'algorithme explore plus facilement et reste beaucoup moins bloqué dans des zones d'allers-retours entre 2 blocs. Toutefois, les résultats ne sont pas parfaits et il réussit rarement à récupérer toutes les récompenses positives.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***